In [1]:
import pickle

from dataset import NewsDataset
from BertModel import WeightedBertForSequenceClassification
from BertModel import Bert_cutted_cosine


from smooth_gradient import SmoothGradient
from integrated_gradient import IntegratedGradient

import torch
from torch import nn
from torch.utils.data import DataLoader
from transformers import DistilBertConfig, DistilBertTokenizer
import pandas as pd
from IPython.display import display, HTML
import numpy as np
from transformers import BertConfig, BertTokenizer
import os

In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


In [3]:
criterion = nn.CrossEntropyLoss()

In [4]:
# cavs_dir='../torch_tcav/tcav_scripts/tcav_class_test_halfsplitFINALWeightedBlueBert512NicuLosNoneRemovedHalfValTest_1_checkpoint-2375_layer11/cavs/'
# f1s_dct={}
# for fn in os.listdir(cavs_dir):
#     with open(os.path.join(cavs_dir,fn),'rb') as file:
#         cav_dct = pickle.load(file)
#     f1=float(cav_dct['metrics'].split()[12])
#     f1s_dct[fn]=f1
with open('../torch_tcav/tcav_scripts/tcav_class_test_halfsplitFINALWeightedBlueBert512NicuLosNoneRemovedHalfValTest_1_checkpoint-2375_layer11/cavs/positive_7742-negative500_7742_22-11-linear-0.1.pkl','rb') as file:
    cav_dct = pickle.load(file)

cav = cav_dct['cavs'][0]

In [5]:
# jaundice_f1s={k:v for k,v in f1s_dct.items() if '7742' in k}
# max(jaundice_f1s, key=jaundice_f1s.get)

In [6]:
print(cav_dct['metrics'])

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       165
           1       0.84      0.82      0.83       165

    accuracy                           0.84       330
   macro avg       0.84      0.84      0.84       330
weighted avg       0.84      0.84      0.84       330



In [7]:
model_path = '../torch_tcav/model_train_scripts/checkpoints/FINALWeightedBlueBert512NicuLosNoneRemovedHalfValTest_1/checkpoint-2375'
bert = WeightedBertForSequenceClassification.from_pretrained(model_path,output_attentions=True,cav=cav,layer_number=12)

Some weights of the model checkpoint at ../torch_tcav/model_train_scripts/checkpoints/FINALWeightedBlueBert512NicuLosNoneRemovedHalfValTest_1/checkpoint-2375 were not used when initializing WeightedBertForSequenceClassification: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing WeightedBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WeightedBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WeightedBertForSequenceClassification were not initialized from the model checkpoint at ../torch_tcav/model_train_scripts/checkpoints/FINALWeightedBlueBert512NicuLosNoneRemovedHalfValTest_1/checkpoint-2375

In [8]:
tokens = tokenizer.encode(['This is a test','so is this'], padding='max_length', max_length=512)
outs=bert(input_ids=torch.LongTensor([tokens]),output_hidden_states=True,return_dict=True,return_logits=False)

In [9]:
outs['hidden_states'][12].detach().cpu().numpy().reshape(1,-1)

array([[ 0.12567417, -0.00343427,  0.5475866 , ..., -0.52313054,
         0.06466903, -0.96396595]], dtype=float32)

In [10]:
outs['logits']

tensor([[0.5008, 0.4992]], grad_fn=<ViewBackward>)

In [11]:
bert.classifier.to(bert.bert.device)
bert.classifier.cav_tensor=bert.classifier.cav_tensor.to(bert.bert.device)

In [12]:
quotes_list = ['IUI, trichorionic, triamniotic','Antepartum remarkable for ART conception with subsequent twinning of one embryo',
              'Growth better than siblings, but still only around 10-25%','Infant TRIPLET BORN AT 31 WEEKS GESTATION']

In [13]:
# grads_df['ROW_ID'] = list(train_df['ROW_ID'])+list(val_df['ROW_ID'])+list(test_df['ROW_ID'])
# grads_df['TEXT'] = list(train_df['TEXT'])+list(val_df['TEXT'])+list(test_df['TEXT'])

In [38]:
texts=['this patient is very healthy','this patient has jaundice']
config = BertConfig()
batch_size=1
all_instances = []
for text in texts:
    test_example = [[text],['']]

    test_dataset = NewsDataset(
        data_list=test_example,
        tokenizer=tokenizer,
        max_length=config.max_position_embeddings, 
    )

    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        shuffle=False,
    )

    integrated_grad = IntegratedGradient(
        bert, 
        criterion, 
        tokenizer, 
        show_progress=True,
        encoder="bert"
    )
    instances = integrated_grad.saliency_interpret(test_dataloader)
    all_instances.append(instances)

100%|██████████| 1/1 [00:03<00:00,  3.86s/it]


In [40]:
coloder_string = integrated_grad.colorize({k:v[:-1] if type(v)==list else v for k,v in all_instances[0][0].items()})
display(HTML(coloder_string))

In [30]:
all_instances[1]

[{'tokens': ['[CLS]',
   'this',
   'patient',
   'is',
   'a',
   'triple',
   '##t',
   'with',
   'ja',
   '##und',
   '##ice',
   'and',
   'the',
   'weight',
   'is',
   'fine',
   'and',
   'siblings',
   'are',
   'also',
   'fine',
   '.',
   '[SEP]'],
  'grad': [0.01634272187948227,
   0.03272367641329765,
   0.02459438145160675,
   0.014008548110723495,
   0.01997995190322399,
   0.11511045694351196,
   0.035266708582639694,
   0.00711166812106967,
   0.0710081085562706,
   0.024661462754011154,
   0.05291587486863136,
   0.004441768862307072,
   0.0391855426132679,
   0.04922395199537277,
   0.020197290927171707,
   0.007337586022913456,
   0.05729655176401138,
   0.042740028351545334,
   0.03723704069852829,
   0.009259200654923916,
   0.02745366096496582,
   0.03163047134876251,
   0.12183572351932526],
  'label': 1,
  'prob': 0.5070853233337402}]

In [51]:
instances[0]

{'tokens': ['[CLS]',
  'this',
  'is',
  'a',
  'test',
  'in',
  'which',
  'the',
  'the',
  'pregnancy',
  'were',
  'under',
  '##weight',
  'triple',
  '##ts',
  'and',
  'pneumonia',
  'and',
  'ja',
  '##und',
  '##ice',
  '.',
  '[SEP]'],
 'grad': [0.01483505591750145,
  0.0071844058111310005,
  0.0031794332899153233,
  0.006486645899713039,
  0.013977427035570145,
  0.0027292438317090273,
  0.009400671347975731,
  0.026495056226849556,
  0.021737011149525642,
  0.009523723274469376,
  0.02988959290087223,
  0.024764586240053177,
  0.020968571305274963,
  0.07252927869558334,
  0.05242902785539627,
  0.036799315363168716,
  0.01895962469279766,
  0.04123625159263611,
  0.0461561493575573,
  0.03622741997241974,
  0.08125228434801102,
  0.042408913373947144,
  0.24532140791416168],
 'label': 1,
 'prob': 0.7216453552246094}

In [1]:
import pandas as pd

In [8]:
# grads_df = pd.read_csv('all_nicu_grads_CAV_positive_V3401-negative500_V3401_40-11-linear-0.1.pkl.csv')
grads_df = pd.read_csv('all_nicu_grads_CAV_positive_7742-negative500_7742_22-11-linear-0.1.pkl.csv')

grads_df['tokens']=grads_df['tokens'].apply(eval)

grads_df['grad'] = grads_df['grad'].apply(eval)

In [9]:
def unravel(lst):
    return [i for j in lst for i in j]

In [10]:
# label = 1
all_class_grads = pd.DataFrame(  list(zip( unravel(grads_df['tokens']),  unravel(grads_df['grad']) )) , columns=['tokens', 'grad']  )

In [11]:
word =''
score=0
words=[]
scores=[]
for i in reversed(range(len(all_class_grads))):
    token=all_class_grads['tokens'].iloc[i]
    if token.startswith('##'):
        word = token[2:]+word
        score = max(score,all_class_grads['grad'].iloc[i] )
#         print(word)
    else:
        word = token+word
#         print(word)
        words.append(word)
        score = max(score,all_class_grads['grad'].iloc[i] )
        scores.append(score)
        word=''
        score=0

In [12]:
tokens_grouped = pd.DataFrame(list(zip(words, scores)), columns=['words', 'grad']).groupby('words')
means = tokens_grouped['grad'].mean()
cts = tokens_grouped['grad'].count()
wrds = tokens_grouped.apply(lambda group: str(group.name))
class_df = pd.DataFrame(list(zip(wrds,means,cts)), columns=['word','grad_mean','word_count' ]).sort_values(by='grad_mean',ascending=False)
class_df = class_df[class_df['word_count']>5]
print(len(class_df))
# print(class_df.head(25).to_latex(index=False, float_format="%.3f"))
class_df.head(50)

5759


,word,grad_mean,word_count
7160,[SEP],0.069539,7597
13268,hbsab,0.038641,6
8575,babycares,0.032463,26
7595,agpars,0.032443,18
3295,300mmhg,0.025502,6
21169,tcpo2,0.023884,6
21133,tags,0.020383,114
15665,minutes,0.019389,1564
21173,teaching,0.017830,19
21166,tco2,0.017804,16


In [24]:
word =''
score=0
negative_words=[]
negative_scores=[]
for i in reversed(range(len(all_negative_grads))):
    token=all_negative_grads['tokens'].iloc[i]
    if token.startswith('##'):
        word = token[2:]+word
        score = max(score,all_negative_grads['grad'].iloc[i] )
#         print(word)
    else:
        word = token+word
#         print(word)
        negative_words.append(word)
        score = max(score,all_negative_grads['grad'].iloc[i] )
        negative_scores.append(score)
        word=''
        score=0

tokens_grouped = pd.DataFrame(list(zip(negative_words, negative_scores)), columns=['words', 'grad']).groupby('words')
means = tokens_grouped['grad'].mean()
cts = tokens_grouped['grad'].count()
wrds = tokens_grouped.apply(lambda group: str(group.name))
negative_df = pd.DataFrame(list(zip(wrds,means,cts)), columns=['word','grad_mean','word_count' ]).sort_values(by='grad_mean',ascending=False)
negative_df = negative_df[negative_df['word_count']>5]
print(negative_df.head(25).to_latex(index=False, float_format="%.3f"))

NameError: name 'all_negative_grads' is not defined